In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medquad/medquad.csv


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [4]:
import os
import torch
import pandas as pd
import json
import warnings
from datasets import load_dataset
from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import evaluate

2024-04-24 04:50:09.440263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 04:50:09.440366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 04:50:09.576874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
test_dataset = load_dataset("prsdm/MedQuad-phi2-1k", split="train[:200]")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
# Assuming `dataset` is your dataset containing the 'text' feature
instructions_list = []

for row in test_dataset['text']:
    # Split the text based on the '### Assistant:' keyword
    parts = row.split('### Assistant:')
    
    # Extract the instruction part (assuming it's before '### Assistant:')
    if len(parts) > 0:
        instruction = parts[0].strip()
        instructions_list.append(instruction)

# Print the extracted instructions
for instruction in instructions_list:
    print(instruction)



### Instruction: How to prevent Lung Cancer ?
### Instruction: What are the treatments for Primary CNS Lymphoma ?
### Instruction: What are the stages of Vulvar Cancer ?
### Instruction: How to diagnose Pancreatic Neuroendocrine Tumors (Islet Cell Tumors) ?
### Instruction: What are the treatments for Parathyroid Cancer ?
### Instruction: What are the stages of Testicular Cancer ?
### Instruction: What are the stages of Male Breast Cancer ?
### Instruction: What are the treatments for Small Intestine Cancer ?
### Instruction: What are the treatments for Gallbladder Cancer ?
### Instruction: What are the treatments for Myelodysplastic/ Myeloproliferative Neoplasms ?
### Instruction: What are the treatments for AIDS-Related Lymphoma ?
### Instruction: What are the treatments for Transitional Cell Cancer of the Renal Pelvis and Ureter ?
### Instruction: What are the stages of Colon Cancer ?
### Instruction: What causes Urinary Retention ?
### Instruction: What are the treatments for Diabe

In [ ]:
perplexity = evaluate.load("perplexity", module_type="metric")
input_texts = instructions_list
results = perplexity.compute(model_id='Sidsky08/Llama-2-7b-chat-finetune17k',
                             add_start_token=False,
                             predictions=input_texts)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]